Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.

In [0]:
%run "/Users/sarfaraj_ahmed@outlook.com/Clean"

In [0]:
df_pin.createOrReplaceTempView("pin")
df_geo.createOrReplaceTempView("geo")
df_user.createOrReplaceTempView("user")

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW users_joined_2015_2020 AS
SELECT
  u.ind,
  EXTRACT(YEAR FROM u.date_joined) AS post_year,
  u.age,
  p.follower_count
FROM
  user u
INNER JOIN 
  pin p
ON
  u.ind = p.ind
WHERE
  EXTRACT(YEAR FROM u.date_joined) BETWEEN 2015 AND 2020
  AND
  p.follower_count IS NOT NULL 
  AND 
  p.follower_count != 0;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW age_groups_users AS
SELECT
  ind,
  post_year,
  follower_count,
  CASE
    WHEN age BETWEEN 18 AND 24 THEN '18-24'
    WHEN age BETWEEN 25 AND 35 THEN '25-35'
    WHEN age BETWEEN 36 AND 50 THEN '36-50'
    WHEN age > 50 THEN '+50'
  END AS age_group
FROM
  users_joined_2015_2020;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW ranked_followers AS
SELECT
  age_group,
  follower_count,
  post_year,
  ROW_NUMBER() OVER (PARTITION BY age_group ORDER BY follower_count) AS row_num,
  COUNT(*) OVER (PARTITION BY age_group) AS total_count
FROM
  age_groups_users;

In [0]:
%sql
SELECT
  age_group,
  AVG(post_year) AS post_year,
  CASE
        WHEN total_count % 2 != 0 THEN MAX(follower_count)
        ELSE AVG(follower_count)
    END AS median_follower_count
FROM
  ranked_followers
WHERE 
  row_num IN ((total_count + 1) / 2, total_count / 2 + 1)
GROUP BY 
  age_group, total_count
ORDER BY
  CASE
    WHEN age_group = '18-24' THEN 1
    WHEN age_group = '25-35' THEN 2
    WHEN age_group = '36-50' THEN 3
    WHEN age_group = '+50' THEN 4
  END;